# 数据集称为“二十新闻组”。这是官方的说明，从网站引用：
#### 20个新闻组数据集是大约20,000个新闻组文档的集合，在20个不同的新闻组中被平均分配（几乎）。
#### 据我们所知，它最初是由肯朗收集的，可能是他的论文“Newsweeder：学习过滤网络新闻”，虽然他没有明确提及这个集合。
#### 20个新闻组集合已成为机器学习技术文本应用中的实验的流行数据集，如文本分类和文本聚类。
#### 在下面我们将使用内置的数据集加载器来从scikit学习的20个新闻组。
#### 或者，可以从网站手动下载数据集，并将该sklearn.datasets.load_files 功能指向20news-bydate-train未压缩归档文件夹的子文件夹。

In [2]:
# 获取数据集中的20个可用数据集中仅使用4个类别的部分数据集：
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian','comp.graphics', 'sci.med']
# 我们现在可以加载与这些类别匹配的文件列表，如下所示：
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

In [41]:
# 返回的数据集是一个scikit-learn“束”：一个简单的持有者对象，其字段可以作为python dict 键或object属性访问，
# 以方便起见，例如 target_names保存所请求的类别名称的列表：
print(twenty_train.target_names)
# 文件本身被加载到data属性的内存中。作为参考，文件名也可用：
print("twenty_train.data:",len(twenty_train.data))
print("twenty_train.filenames",len(twenty_train.filenames))
print(twenty_train.filenames)

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']
twenty_train.data: 2257
twenty_train.filenames 2257
[ 'C:\\Users\\lenovo\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.graphics\\38440'
 'C:\\Users\\lenovo\\scikit_learn_data\\20news_home\\20news-bydate-train\\comp.graphics\\38479'
 'C:\\Users\\lenovo\\scikit_learn_data\\20news_home\\20news-bydate-train\\soc.religion.christian\\20737'
 ...,
 'C:\\Users\\lenovo\\scikit_learn_data\\20news_home\\20news-bydate-train\\sci.med\\58112'
 'C:\\Users\\lenovo\\scikit_learn_data\\20news_home\\20news-bydate-train\\sci.med\\58578'
 'C:\\Users\\lenovo\\scikit_learn_data\\20news_home\\20news-bydate-train\\sci.med\\58895']


In [15]:
# 我们打印第一个加载的文件的第一行：
print("\n".join(twenty_train.data[0].split("\n")[:3]))
# 受监督的学习算法将需要训练集中的每个文档的类别标签。在这种情况下，类别是新闻组的名称，它也恰好是保存单个文档的文件夹的名称。
# 对于速度和空间效率，scikit-learn将目标属性加载为与target_names列表中类别名称的索引对应的整数数组。每个样本的类别整数ID存储在target属性中：
print("\n",twenty_train.target_names[twenty_train.target[0]])
print(twenty_train.target[:10])
# 可以如下回到类别名称：
for t in twenty_train.target[:10]:
     print(twenty_train.target_names[t])

From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton

 comp.graphics
[1 1 3 3 3 3 3 2 2 2]
comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


# 用表征文本scikit-learn

#### 文本预处理，令牌化和过滤的无效词包含在高级组件中，能够构建特征字典并将文档转换为特征向量：

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
print(X_train_counts.shape)
# CountVectorizer支持N克单词或连续字符的计数。一旦装配，矢量化程序已经建立了一个特征索引字典：
count_vect.vocabulary_.get(u'algorithm')

(2257, 35788)


4690

# 从出现到频率

#### 发生次数是一个很好的开始，但是有一个问题：更长的文档将具有比较短的文档更高的平均值，即使他们可能会讨论相同的主题。
#### 为了避免这些潜在差异，将文档中每个单词的出现次数除以文档中的单词总数就足够了：这些新功能被称为tf“术语频率”。
#### 在tf之上的另一个改进是对在语料库中的许多文档中发生的词的缩减权重，因此比仅在较小部分语料库中出现的词语的信息量更少。
#### 这个缩减被称为tf-idf，用于“Term Frequency times Inverse Document Frequency”。
#### 两个TF和TF-IDF可以计算如下：

In [23]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
print(X_train_tf.shape)

# 在上面的示例代码中，我们首先使用该fit(..)方法将我们的估计器与数据进行匹配，
#其次是transform(..)将我们的计数矩阵转换为tf-idf表示法的方法。
#通过跳过冗余处理，可以组合这两个步骤，以更快地实现相同的最终结果。
#这是通过使用fit_transform(..)如下所示的方法完成的 ，如上一节中的注释所述：
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print(X_train_tfidf.shape)

(2257, 35788)
(2257, 35788)


# 训练分类器

#### 现在我们有了我们的功能，我们可以训练分类器来尝试预测一个帖子的类别。
#### 让我们从一个天真的贝叶斯 分类器开始，为这个任务提供了一个很好的基准。
#### scikit-learn包括该分类器的几个变体; 最适合单词计数的是多项式：

In [25]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)
# 为了尝试在新文档中预测结果，我们需要使用与以前几乎相同的特征提取链来提取特征。
# 不同的是，我们所说transform的，而不是fit_transform 对变压器，因为它们已经适应训练集：
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, twenty_train.target_names[category]))

'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


#### 构建管道
#### 为了使vectorizer => transform => classifier更容易使用，scikit-learn提供了一个Pipeline类似复合分类器的类：

In [28]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),])
text_clf.fit(twenty_train.data, twenty_train.target)  

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

### 测试集上的性能评估
#### 评估模型的预测准确度：

In [29]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
     categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)      

0.83488681757656458

#### 让我们看看我们是否可以用线性支持向量机（SVM）来做得更好，
#### 这是被广泛认为是最好的文本分类算法之一（尽管它也比初学贝叶斯慢一些）。
#### 我们可以通过将不同的分类对象插入到我们的管道中来改变学习者：

In [30]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),
                      ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42,
                                            max_iter=5, tol=None)),])
text_clf.fit(twenty_train.data, twenty_train.target)  

predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)   

0.9127829560585885

#### 进一步提供实用程序以更详细地对结果进行性能分析：

In [31]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))
metrics.confusion_matrix(twenty_test.target, predicted)

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]], dtype=int64)

### 参数整定使用格搜索
#### 我们已经遇到了一些参数，如use_idf在 TfidfTransformer。分类器也有许多参数; 例如，MultinomialNB包括平滑参数alpha， 并且在目标函数中SGDClassifier具有惩罚参数alpha和可配置的损失和惩罚项（参见模块文档或使用Python help函数来获取这些描述）。
#### 代替调整链的各种组件的参数，可以对可能值的网格上的最佳参数进行详尽的搜索。我们尝试使用单词或双字母的所有分类器，具有或不具有idf，对于线性SVM，可以使用0.01或0.001的惩罚参数：

In [40]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3),}
# 显然，这种详尽的搜索可能是昂贵的。如果我们拥有多个CPU内核，我们可以告诉电网搜索者与参数并行地尝试这八个参数组合n_jobs。
# 如果我们给这个参数一个值-1，网格搜索将检测安装了多少个核心并使用它们全部：
gs_clf = GridSearchCV(text_clf, parameters, n_jobs=-1)
#print(gs_clf)
# 网格搜索实例的行为与普通scikit-learn 模型类似。让我们在训练数据的较小子集上进行搜索，以加速计算：
gs_clf= gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])
#print(gs_clf)
# 调用的结果fit一上GridSearchCV对象的分类，我们可以用它来predict：
print(twenty_train.target_names[gs_clf.predict(['God is love'])[0]])
# 对象best_score_和best_params_属性存储与该分数相对应的最佳平均分数和参数设置：
gs_clf.best_score_                                  
for param_name in sorted(parameters.keys()):
     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print(gs_clf.cv_results_)

soc.religion.christian
clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)
{'mean_train_score': array([ 0.99374372,  1.        ,  0.94123886,  0.97623272,  1.        ,
        1.        ,  0.98499057,  1.        ]), 'param_vect__ngram_range': masked_array(data = [(1, 1) (1, 2) (1, 1) (1, 2) (1, 1) (1, 2) (1, 1) (1, 2)],
             mask = [False False False False False False False False],
       fill_value = ?)
, 'mean_score_time': array([ 0.0496695 ,  0.11267312,  0.09800577,  0.21101205,  0.06167014,
        0.12067358,  0.07633766,  0.11934018]), 'std_train_score': array([ 0.00355103,  0.        ,  0.00779216,  0.00989579,  0.        ,
        0.        ,  0.01061333,  0.        ]), 'rank_test_score': array([3, 4, 8, 6, 1, 2, 7, 5]), 'std_fit_time': array([ 0.01892782,  0.08209399,  0.20419503,  0.10186309,  0.02758173,
        0.3004017 ,  0.34428207,  0.01564272]), 'split2_test_score': array([ 0.89473684,  0.90225564,  0.81203008,  0.81203008,  0.93984962,
        0.